In [1]:
import os
import tempfile
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.8.2


download the model

In [4]:
import wget

In [6]:
MODEL_NAME = 'mobilenetv2_dm05_coco_voc_trainval'  # @param ['mobilenetv2_dm05_coco_voc_trainval', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainval']

DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
MODEL_URLS = {
    'mobilenetv2_dm05_coco_voc_trainval':
        'deeplabv3_mnv2_dm05_pascal_trainval_2018_10_01.tar.gz',
    'mobilenetv2_coco_voctrainval':
        'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
    'xception_coco_voctrainval':
        'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
}

MODEL_TAR = MODEL_URLS[MODEL_NAME]
MODEL_URL = DOWNLOAD_URL_PREFIX + MODEL_TAR
print(MODEL_TAR, MODEL_URL)

deeplabv3_mnv2_dm05_pascal_trainval_2018_10_01.tar.gz http://download.tensorflow.org/models/deeplabv3_mnv2_dm05_pascal_trainval_2018_10_01.tar.gz


In [7]:
wget.download(MODEL_URL, MODEL_TAR)

 22% [................                                                          ] 1785856 / 7976055

100% [..........................................................................] 7976055 / 7976055

'deeplabv3_mnv2_dm05_pascal_trainval_2018_10_01.tar.gz'

unzip (manually with 7zip) and get the file location

In [2]:
MODEL_FILE = "deeplabv3_mnv2_dm05_pascal_trainval/frozen_inference_graph.pb"
print('Frozen graph file path:', MODEL_FILE)

Frozen graph file path: deeplabv3_mnv2_dm05_pascal_trainval/frozen_inference_graph.pb


the output tensor is resizebilinear_2

<b>no need to run this part</b>

see https://colab.research.google.com/github/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/DeepLabV3/DeepLab_TFLite_COCO.ipynb

In [9]:
# Load the TensorFlow model
# The preprocessing and the post-processing steps should not be included in the TF Lite model graph 
# because some operations (ArgMax) might not support the delegates. 
# Insepct the graph using Netron https://lutzroeder.github.io/netron/
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file = MODEL_FILE, 
    input_arrays = ['sub_7'],
    output_arrays = ['ResizeBilinear_2']
)

# Optional: Perform the simplest optimization known as post-training dynamic range quantization.
# https://www.tensorflow.org/lite/performance/post_training_quantization#dynamic_range_quantization
# You can refer to the same document for other types of optimizations.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert to TFLite Model
tflite_model = converter.convert()

_, dynamic_tflite_path = tempfile.mkstemp('.tflite')
tflite_model_size = open(dynamic_tflite_path, 'wb').write(tflite_model)
tf_model_size = os.path.getsize(MODEL_FILE)
print('TensorFlow Model is  {} bytes'.format(tf_model_size))
print('TFLite Model is      {} bytes'.format(tflite_model_size))
print('Post training dynamic range quantization saves {} bytes'.format(tf_model_size-tflite_model_size))

TensorFlow Model is  3042785 bytes
TFLite Model is      936592 bytes
Post training dynamic range quantization saves 2106193 bytes


run the following command in ternminal to convert the model

<b>do not run this in jupyter</b>

see https://www.tensorflow.org/js/tutorials/conversion/import_saved_model

In [ ]:
tensorflowjs_converter --input_format=tf_frozen_model --output_node_names='ResizeBilinear_2' deeplabv3_mnv2_dm05_pascal_trainval/frozen_inference_graph.pb deeplabv3_mnv2_dm05_pascal_trainval/web